In [217]:
from matplotlib import pyplot as plt
from matplotlib import colors as colors
import numpy as np

import sys 
sys.path.append('../')
import Utils.utils as util
from importlib import reload
reload(util)

from sklearn.linear_model import Lasso
from sklearn.multiclass import OneVsRestClassifier

In [257]:
def convert_patch_into_vector(patch):
    h = patch.shape[0]
    #Convert one patch into a column vector for training
    patch = patch.flatten()
    train_y = patch[(h*(h//2)+(h//2))*3:(h*(h//2)+(h//2))*3+3]
    train_x = np.delete(patch, range((h*(h//2)+(h//2))*3, (h*(h//2)+(h//2))*3+3))
    return np.array((train_x, train_y))
    
def convert_vector_into_patch(vector, h):
    #Convert a one column vector into a patch of size h
    return vector.reshape((h,h,3))
    
def convert_patches_into_training_set(patches):
    #Convert a set of patches into a set of vectors
    return map(convert_patch_into_vector, clear_patches)



In [252]:
img_matrice = util.read_im("Data/imgdataset/space.jpg")

print("taille de l'image : ", img_matrice.shape)

#noisy_image = delete_rect(img_matrice, 400, 400, 500, 100)
#display_im(noisy_image)

#display_im(img_matrice)

#util.display_im(util.get_patch(50, 50, 4, img_matrice))

#noisy_image = util.noise(img_matrice, 0.1)

patches = util.get_patches(noisy_image, 3, img_matrice.shape[0], img_matrice.shape[1])

noisy_patches = patches[0]
clear_patches = patches[1]

#display_im(noisy_image)
#print("New patch : ", convert_patch_into_vector(clear_patches[5]))


taille de l'image :  (256, 256, 3)


In [258]:
training = convert_patches_into_training_set(patches)

a, b = zip(*training)
a = np.array(a)



In [259]:
label = a.flatten()

label = label.T

training = a.T

print(label.shape)
print(training.shape)

print("--- lasso regression ---")
classif = OneVsRestClassifier(Lasso(alpha=0.001, fit_intercept=True))
classif.fit(training, label)

(599184,)
(24, 24966)
--- lasso regression ---


ValueError: Found input variables with inconsistent numbers of samples: [24, 599184]